In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
import pprint as pp

In [2]:
def get_url_info():
    dept = input("Departure : ")
    arr = input("Arrival : ")
    date = input("Date : ")
    type_train = input("Type Train : ")
    if (type_train != "oneway"):
        ret_date = input("Return date : ")
    adult = input("Adult : ")
    infant = input("Infant : ")
    if (type_train != "oneway"):
        url = ("https://m.tiket.com/kereta-api/cari?d=" + dept + "&dt=STATION&a=" + arr + "&at=STATION&date=" + date +"&typetrain=" + type_train + "&ret_date=" + ret_date + "&adult=" + adult + "&infant=" + infant)
    else:
        url = ("https://m.tiket.com/kereta-api/cari?d=" + dept + "&dt=STATION&a=" + arr + "&at=STATION&date=" + date +"&typetrain=" + type_train + "&adult=" + adult + "&infant=" + infant)
    return url
    
def scrape_small_picture(url):
#     url = get_url_info()
#     print(url)
    result = []
    page = urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')
    data_names = ['data-origin_station', 'data-destination_station', 'data-departure_time', 'data-train_class', 'data-price', 'data-duration', 'data-train_name']
    rows = soup.find_all("tr", {'class' : 'main-row '})
    for row in rows:
        row_datas = [row.get(data) for data in data_names]
        result.append(row_datas)
    return result

def make_dict(datas, day, id):
    result = {}
    result["ticket"] = []
    for data in datas:
        ticket = dict()
        ticket["departure"] = data[0]
        ticket["arrival"] = data[1]
        ticket["hour"] = data[2]
        ticket["type_train"] = data[3]
        ticket["adult"] = data[4]
        ticket["infant"] = data[5]
        ticket["month"] = day.month
        ticket["date"] = day.day
        ticket["weekday"] = day.weekday()
        ticket["day"] = day.strftime("%A")
        ticket["datetime"] = day
        result["id"] = id
        result["ticket"].append(ticket)
    return result

In [3]:
# main

def scrape_big_picture(days):
    datas = []
    stations = ['GMR']

    id = 0

    for station in stations:
        ret_stations = ['BD', 'YK', 'SGU']
        for ret_station in ret_stations:
            for day in days:
                if day.day < 10:
                    if day.month < 10:
                        url = 'https://m.tiket.com/kereta-api/cari?d=' + station + '&dt=STATION&a=' + ret_station + '&at=STATION&date=2018-0' + str(day.month) + '-0' + str(day.day) + '&typetrain=oneway&adult=1&infant=0'
                        datas.append(make_dict(scrape_small_picture(url), day, id))
                    else:
                        url = 'https://m.tiket.com/kereta-api/cari?d=' + station + '&dt=STATION&a=' + ret_station + '&at=STATION&date=2018-' + str(day.month) + '-0' + str(day.day) + '&typetrain=oneway&adult=1&infant=0'
                        datas.append(make_dict(scrape_small_picture(url), day, id))
                else:
                    if day.month < 10:
                        url = 'https://m.tiket.com/kereta-api/cari?d=' + station + '&dt=STATION&a=' + ret_station + '&at=STATION&date=2018-0' + str(day.month) + '-' + str(day.day) + '&typetrain=oneway&adult=1&infant=0'
                        datas.append(make_dict(scrape_small_picture(url), day, id))
                    else:
                        url = 'https://m.tiket.com/kereta-api/cari?d=' + station + '&dt=STATION&a=' + ret_station + '&at=STATION&date=2018-0' + str(day.month) + '-' + str(day.day) + '&typetrain=oneway&adult=1&infant=0'
                        datas.append(make_dict(scrape_small_picture(url), day, id))
                id += 1
                    
    return datas

In [4]:
import datetime

today = datetime.datetime.today()
days = []

for x in range(0,90):
    days.append(today)
    today += datetime.timedelta(1)

In [6]:
# main
datas = scrape_big_picture(days)


In [19]:
import pandas as pd

for data in datas:
    if data['ticket'] == []:
        datas.remove(data)

In [21]:
import pandas as pd

for data in datas:
    if data['ticket'] == []:
        print(data['ticket'])

In [22]:
df = pd.concat(
    [
        pd.concat([pd.Series(m) for m in t['ticket']], axis=1) for t in datas
    ], keys=[t['id'] for t in datas]
)

df.index.levels[0].name = 'id'
df.columns.name = 'ticket'

df = df.T.stack(0).swaplevel(0, 1).sort_index().reset_index()

In [23]:
df.to_csv("jkt-bdg.csv", index=False)

In [24]:
df1 = pd.read_csv("jkt-bdg.csv")
df1

,id,ticket,adult,arrival,date,datetime,day,departure,hour,infant,month,type_train,weekday
0,1,0,90000,BD,8,2018-08-08 22:15:02.134755,Wednesday,GMR,750,11820,8,eko,2
1,1,1,135000,BD,8,2018-08-08 22:15:02.134755,Wednesday,GMR,1130,12180,8,eks,2
2,1,2,100000,BD,8,2018-08-08 22:15:02.134755,Wednesday,GMR,1130,12180,8,eko,2
3,1,3,125000,BD,8,2018-08-08 22:15:02.134755,Wednesday,GMR,1235,12300,8,eks,2
4,1,4,120000,BD,8,2018-08-08 22:15:02.134755,Wednesday,GMR,1405,12900,8,eks,2
5,1,5,80000,BD,8,2018-08-08 22:15:02.134755,Wednesday,GMR,1530,11340,8,eko,2
6,1,6,90000,BD,8,2018-08-08 22:15:02.134755,Wednesday,GMR,1800,11640,8,eko,2
7,1,7,80000,BD,8,2018-08-08 22:15:02.134755,Wednesday,GMR,1845,11340,8,eko,2
8,2,0,125000,BD,9,2018-08-09 22:15:02.134755,Thursday,GMR,505,12840,8,eks,3
9,2,1,80000,BD,9,2018-08-09 22:15:02.134755,Thursday,GMR,505,12840,8,eko,3


In [27]:
for t in df1.hour:
    print(type(t))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class